# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "af" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [3]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-af-baseline


In [4]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 9.3MB/s 


In [0]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz

In [5]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2019-11-26 11:15:28--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.005s  

2019-11-26 11:15:29 (52.1 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2019-11-26 11:15:31--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-af.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [6]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [7]:
import pandas as pd

# TMX file to dataframe
source_file = 'enaf_parallel.train.' + source_language
target_file = 'enaf_parallel.train.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 1/50171 lines since contained in test set.


,source_sentence,target_sentence
0,( 3 ) if material transfer agreements or benef...,( 3 ) indien materiaaloordragooreenkomste of v...
1,( a ) signed material transfer agreements or b...,( a ) getekende materiaaloordragooreenkomste o...
2,( b ) if it has not been possible to conclude ...,( b ) indien dit nie moontlik was om sodanige ...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [8]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144662 sha256=839189df0920c81aeedf1e6572a35a13de649e40fcf0d61080db2673c1e227b1
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.02 0.00 percent complete
00:00:14.44 2.08 percent complete
00:00:28.83 4.17 percent complete
00:00:43.76 6.25 percent complete
00:00:58.65 8.34 percent complete
00:01:13.09 10.42 percent complete
00:01:27.06 12.50 percent complete
00:01:40.52 14.59 percent complete


00:01:54.51 16.67 percent complete
00:02:08.22 18.75 percent complete
00:02:21.96 20.84 percent complete
00:02:36.32 22.92 percent complete
00:02:50.34 25.01 percent complete
00:03:04.63 27.09 percent complete
00:03:18.58 29.17 percent complete
00:03:32.73 31.26 percent complete


00:03:46.90 33.34 percent complete
00:04:01.33 35.42 percent complete
00:04:15.57 37.51 percent complete
00:04:29.17 39.59 percent complete
00:04:42.97 41.68 percent complete
00:04:57.24 43.76 percent complete
00:05:11.76 45.84 percent complete
00:05:25.78 47.93 percent complete
00:05:39.29 50.01 percent complete
00:05:53.25 52.09 percent complete
00:06:07.30 54.18 percent complete
00:06:21.92 56.26 percent complete
00:06:35.96 58.35 percent complete
00:06:50.12 60.43 percent complete
00:07:04.53 62.51 percent complete
00:07:18.47 64.60 percent complete


00:07:32.24 66.68 percent complete
00:07:46.07 68.76 percent complete
00:07:59.82 70.85 percent complete
00:08:13.33 72.93 percent complete
00:08:26.94 75.02 percent complete
00:08:41.12 77.10 percent complete
00:08:55.04 79.18 percent complete
00:09:09.58 81.27 percent complete
00:09:23.75 83.35 percent complete
00:09:37.88 85.43 percent complete
00:09:52.05 87.52 percent complete
00:10:06.92 89.60 percent complete
00:10:21.68 91.69 percent complete
00:10:35.72 93.77 percent complete
00:10:50.09 95.85 percent complete
00:11:04.32 97.94 percent complete


In [10]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.af <==
as persone aan ' n oortreding ingevolge dié verordening skuldig bevind word , welke oortreding skade aan of verlies van eiendom veroorsaak het en / of ' n nadelige uitwerking op die omgewing gehad het , kan die hof dan , benewens enige ander vonnis wat opgelê is , die volgende doen :
* Small Claims Courts Act (Wet) (File type: pdf; size: 487 KB) (net in Engels beskikbaar)
* Kinders wat min urineer
Is Berading Nodig Voordat Mens 'n Vigstoets Ondergaan?
* Mei/Junie 2006, Vol. 2, Issue 9
Die regering moet seker maak dat mense toegang het tot behoorlike behuising.
Dit is balju's se plig om alle mense, ryk en arm, te alle tye met empatie en waardigheid te behandel.
Tuisblad > Jou Regering > Wes-Kaapse Provinsiale Regering > Wes-Kaap Onderwysdepartement > Fasiliteite > Sentrums vir Basiese Onderwys en Opleiding vir Volwassenes (BOOV), en Gemeenskapsleersentrums > Siyafuna Ukufunda Adult Learning Centre
* Hoe om Aansoek te Doen om 'n Lisensie
Vir gereelde ondersoeke en medika



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [11]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 2199 (delta 4), reused 5 (delta 3), pack-reused 2184
Receiving objects: 100% (2199/2199), 2.60 MiB | 963.00 KiB/s, done.
Resolving deltas: 100% (1525/1525), done.
Processing /content/joeynmt
     |████████████████████████████████| 266kB 39.2MB/s 
     |████████████████████████████████| 307kB 52.2MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 215kB 47.0MB/s 
     |████████████████████████████████| 737kB 52.4MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=724dda0553f12fc3e25ad81a5090e464a3502c77c6529930463c0eba6d4f5066
  Stored in directory: /tmp/pip-ephem-wheel-cache-ipbh1mlc/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyya

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [12]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Afrikaans Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.bpe.en  test.bpe.af  test.en-any.en  train.bpe.en
dev.af		dev.en	    test.bpe.en  train.af	 train.en
dev.bpe.af	test.af     test.en	 train.bpe.af
bpe.codes.4000	dev.bpe.en  test.bpe.af  test.en-any.en  train.bpe.en
dev.af		dev.en	    test.bpe.en  train.af	 train.en
dev.bpe.af	test.af     test.en	 train.bpe.af
BPE Afrikaans Sentences
E@@ k mo@@ es h@@ om om@@ k@@ oop sodat hy sy o@@ ë s@@ ou sluit vir die maatskappy se on@@ eer@@ likheid .
Ge@@ vol@@ g@@ lik was ek beken@@ d as ’ n on@@ eer@@ like man . To@@ e ek in die waar@@ heid kom , het ek gew@@ ei@@ er om daar@@ me@@ e voor@@ t te gaan , al het ek ’ n go@@ eie sal@@ aris ver@@ dien .
E@@ k stel ’ n go@@ eie voorbe@@ eld vir my twee se@@ un@@ s , en ek het voor@@ regte in die gemeen@@ te ge@@ kry .
Bel@@ ast@@ ing@@ ou@@ di@@ te@@ urs en ander met wie ek besi@@ gheid doen , we@@ et n@@ ou dat ek ’ n eer@@ like man is . ”
R@@ ut het na I@@ s@@ ra@@ el ge@@ trek , waar sy die w@@ are G@@ o@@ d k@@ on aan@@ bi@@ d .

In [13]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.bpe.en  test.bpe.af  test.en-any.en  train.bpe.en
dev.af		dev.en	    test.bpe.en  train.af	 train.en
dev.bpe.af	test.af     test.en	 train.bpe.af


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 2000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [15]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2019-11-26 11:31:47,451 Hello! This is Joey-NMT.
2019-11-26 11:31:49,152 Total params: 12138240
2019-11-26 11:31:49,154 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [16]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "/content/drive/My Drive/masakhane/en-af-baseline/models/enaf_transformer/"

cp: cannot create symbolic link '/content/drive/My Drive/masakhane/en-af-baseline/models/enaf_transformer/best.ckpt': Function not implemented


In [17]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 2000	Loss: 74854.25781	PPL: 24.73169	bleu: 12.36405	LR: 0.00030000	*
Steps: 4000	Loss: 52469.36719	PPL: 9.47559	bleu: 26.50927	LR: 0.00030000	*
Steps: 6000	Loss: 42362.46094	PPL: 6.14452	bleu: 33.55846	LR: 0.00030000	*
Steps: 8000	Loss: 37101.53125	PPL: 4.90418	bleu: 37.38237	LR: 0.00030000	*
Steps: 10000	Loss: 34156.69531	PPL: 4.32270	bleu: 39.24070	LR: 0.00030000	*
Steps: 12000	Loss: 32103.47656	PPL: 3.95857	bleu: 40.75340	LR: 0.00030000	*
Steps: 14000	Loss: 30750.45508	PPL: 3.73555	bleu: 42.70344	LR: 0.00030000	*
Steps: 16000	Loss: 29685.38477	PPL: 3.56887	bleu: 43.53423	LR: 0.00030000	*
Steps: 18000	Loss: 28864.55078	PPL: 3.44550	bleu: 44.48371	LR: 0.00030000	*
Steps: 20000	Loss: 28165.52148	PPL: 3.34381	bleu: 45.28709	LR: 0.00030000	*
Steps: 22000	Loss: 27861.96484	PPL: 3.30059	bleu: 45.67252	LR: 0.00030000	*
Steps: 24000	Loss: 27393.80469	PPL: 3.23503	bleu: 47.06020	LR: 0.00030000	*
Steps: 26000	Loss: 26914.48047	PPL: 3.16925	bleu: 47.45031	LR: 0.00030000	*
Steps: 28000	Lo

In [18]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2019-11-26 13:49:10,919 Hello! This is Joey-NMT.
2019-11-26 13:49:45,068  dev bleu:  48.65 [Beam search decoding with beam size = 5 and alpha = 1.0]
2019-11-26 13:50:55,979 test bleu:  19.56 [Beam search decoding with beam size = 5 and alpha = 1.0]
